# OmniSafe Tutorial - Environment Customization

OmniSafe: https://github.com/PKU-Alignment/omnisafe

Documentation: https://omnisafe.readthedocs.io/en/latest/

Safety-Gymnasium: https://www.safety-gymnasium.com/

[Safety-Gymnasium](https://www.safety-gymnasium.com/) is a highly scalable and customizable Safe Reinforcement Learning library, aiming to deliver a good view of benchmarking Safe Reinforcement Learning (Safe RL) algorithms and a more standardized setting of environments. 

## 引言

在本节当中，我们将为您介绍如何在OmniSaef中自定义环境。我们付出了很大的努力将环境接口与OmniSafe的内部逻辑解耦合。您只需要基于我们提供的最小模版作相应改动，即可享受全套的参数定制、算法训练以及结果保存功能。

具体而言，我们将从以下两个部分出发：

- **从OmniSafe的最简环境定制化模版出发** 我们提供了一个用于定制化环境的最简单模版。通过该模版，你将了解：
  - 定制化环境注册。
  - 环境创建参数指定。
  - 环境特定信息记录。

## 快速安装

In [ ]:
# 通过pip安装（如果您已经安装，请忽略此段代码）
!pip install omnisafe

In [ ]:
# 通过源代码安装（如果您已经安装，请忽略此段代码）
## 克隆仓库
!git clone https://github.com/PKU-Alignment/omnisafe
%cd omnisafe

## 完成安装
!pip install -e .

## 定制化环境最简模版
OmniSafe的定制化环境可以仅通过单个文件实现。我们将为您介绍一个最简的定制化环境模版，它将作为您入门的起点。

### 定制化环境设计
我们将在此细致地介绍一个简易随机环境的定制过程。如果您是强化学习领域的专家或有经验的研究者，可以跳过该模块至[定制化环境嵌入](#定制化环境嵌入)

In [1]:
# 导入必要的包
from __future__ import annotations

import random
import omnisafe
from typing import Any, ClassVar

import torch
from gymnasium import spaces

from omnisafe.envs.core import CMDP, env_register, remove_from_register

/home/jiayi/miniconda3/envs/omnisafe/lib/python3.8/site-packages/torch/cuda/__init__.py:141: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 11070). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


In [2]:
# 定义环境类
class ExampleEnv(CMDP):
    _support_envs: ClassVar[list[str]] = ['Example-v0']  # 支持的任务名称

    need_auto_reset_wrapper = True  # 是否需要 `AutoReset` Wrapper
    need_time_limit_wrapper = True  # 是否需要 `TimeLimit` Wrapper

您需要关注上面这段代码的如下细节：

- **任务名称定义** 在 `_support_envs` 中定义环境支持的任务名称。
- **Wrapper配置** 通过设定 `need_auto_reset_wrapper` 和 `need_time_limit_wrapper` 来定义自动重置和限制时间。
- **并行环境数量** 如果您的环境支持向量化并行，请通过 `_num_envs` 参数进行设定。

`AutoReset` Wrapper与 `TimeLimit` Wrapper 的更多信息将在本教程的剩余部分介绍。在此处您只需了解可以通过此接口进行开关即可。

In [3]:
class ExampleEnv(CMDP):
    _support_envs: ClassVar[list[str]] = ['Example-v0', 'Example-v1']  # 支持的任务名称

    need_auto_reset_wrapper = True  # 是否需要 `AutoReset` Wrapper
    need_time_limit_wrapper = True  # 是否需要 `TimeLimit` Wrapper

    def __init__(self, env_id: str, **kwargs) -> None:
        self._count = 0
        self._num_envs = 1
        self._observation_space = spaces.Box(low=-1.0, high=1.0, shape=(3,))
        self._action_space = spaces.Box(low=-1.0, high=1.0, shape=(2,))

完成 `__init__` 函数定义。此处需要指定环境的动作空间与观测空间。你也可以根据具体的任务来定义。例如在本例子中，您可以尝试：
```python
if env_id == 'Example-v0':
    self._observation_space = spaces.Box(low=-1.0, high=1.0, shape=(3,))
    self._action_space = spaces.Box(low=-1.0, high=1.0, shape=(2,))
elif env_id == 'Example-v1':
    self._observation_space = spaces.Box(low=-1.0, high=1.0, shape=(4,))
    self._action_space = spaces.Box(low=-1.0, high=1.0, shape=(3,))
else:
    raise NotImplementedError
```
**请注意，在定制化时请勿修改变量名 `self._observation_space` 以及 `self._action_space`。**

完成环境初始化相关函数的定义。我们认为 `reset` 和 `set_seed` 是环境初始化的必要函数。其中 `reset` 重置环境状态与计步器。而 `set_seed` 通过设定随机种子确保实验的可复现性。实现参考如下：

In [4]:
class ExampleEnv(CMDP):
    _support_envs: ClassVar[list[str]] = ['Example-v0', 'Example-v1']  # 支持的任务名称

    need_auto_reset_wrapper = True  # 是否需要 `AutoReset` Wrapper
    need_time_limit_wrapper = True  # 是否需要 `TimeLimit` Wrapper

    def __init__(self, env_id: str, **kwargs) -> None:
        self._count = 0
        self._num_envs = 1
        self._observation_space = spaces.Box(low=-1.0, high=1.0, shape=(3,))
        self._action_space = spaces.Box(low=-1.0, high=1.0, shape=(2,))

    def set_seed(self, seed: int) -> None:
        random.seed(seed)

    def reset(
        self,
        seed: int | None = None,
        options: dict[str, Any] | None = None,
    ) -> tuple[torch.Tensor, dict]:
        if seed is not None:
            self.set_seed(seed)
        obs = torch.as_tensor(self._observation_space.sample())
        self._count = 0
        return obs, {}

完成功能性函数的定义。`sample_action` 函数用于从环境中随机采样动作；`render` 函数用于渲染环境；`close` 函数用于训练结束后的清理。其中 `sample_action` 是 `off-policy` 和 `offline` 算法必须使用的函数，`render` 和 `close` 是可选的。

In [6]:
class ExampleEnv(CMDP):
    _support_envs: ClassVar[list[str]] = ['Example-v0', 'Example-v1']  # 支持的任务名称

    need_auto_reset_wrapper = True  # 是否需要 `AutoReset` Wrapper
    need_time_limit_wrapper = True  # 是否需要 `TimeLimit` Wrapper

    def __init__(self, env_id: str, **kwargs) -> None:
        self._count = 0
        self._num_envs = 1
        self._observation_space = spaces.Box(low=-1.0, high=1.0, shape=(3,))
        self._action_space = spaces.Box(low=-1.0, high=1.0, shape=(2,))

    def set_seed(self, seed: int) -> None:
        random.seed(seed)

    def reset(
        self,
        seed: int | None = None,
        options: dict[str, Any] | None = None,
    ) -> tuple[torch.Tensor, dict]:
        if seed is not None:
            self.set_seed(seed)
        obs = torch.as_tensor(self._observation_space.sample())
        self._count = 0
        return obs, {}

    def sample_action(self) -> torch.Tensor:
        return torch.as_tensor(self._action_space.sample())

    def render(self) -> Any:
        pass

    def close(self) -> None:
        pass

完成 `step` 函数定义。此处是您定制化环境的核心交互逻辑。您只需按照本例中的数据输入与输出格式进行调整即可。您也可以直接将本例中的随机交互动态更改为您的环境动态。

In [7]:
class ExampleEnv(CMDP):
    _support_envs: ClassVar[list[str]] = ['Example-v0', 'Example-v1']  # 支持的任务名称
    metadata: ClassVar[dict[str, int]] = {}

    need_auto_reset_wrapper = True  # 是否需要 `AutoReset` Wrapper
    need_time_limit_wrapper = True  # 是否需要 `TimeLimit` Wrapper

    def __init__(self, env_id: str, **kwargs) -> None:
        self._count = 0
        self._num_envs = 1
        self._observation_space = spaces.Box(low=-1.0, high=1.0, shape=(3,))
        self._action_space = spaces.Box(low=-1.0, high=1.0, shape=(2,))

    def set_seed(self, seed: int) -> None:
        random.seed(seed)

    def reset(
        self,
        seed: int | None = None,
        options: dict[str, Any] | None = None,
    ) -> tuple[torch.Tensor, dict]:
        if seed is not None:
            self.set_seed(seed)
        obs = torch.as_tensor(self._observation_space.sample())
        self._count = 0
        return obs, {}

    def sample_action(self) -> torch.Tensor:
        return torch.as_tensor(self._action_space.sample())

    def render(self) -> Any:
        pass

    def close(self) -> None:
        pass

    def step(
        self,
        action: torch.Tensor,
    ) -> tuple[torch.Tensor, torch.Tensor, torch.Tensor, torch.Tensor, torch.Tensor, dict]:
        self._count += 1
        obs = torch.as_tensor(self._observation_space.sample())
        reward = 10000 * torch.as_tensor(random.random())
        cost = 10000 * torch.as_tensor(random.random())
        terminated = torch.as_tensor(random.random() > 0.9)
        truncated = torch.as_tensor(self._count > 10)
        return obs, reward, cost, terminated, truncated, {'final_observation': obs}

接下来，我们试着运行该环境10个时间步，观察交互信息。

In [8]:
env = ExampleEnv(env_id='Example-v0')
env.reset(seed=0)
while True:
    action = env.action_space.sample()
    obs, reward, cost, terminated, truncated, info = env.step(action)
    print('-' * 20)
    print(f'obs: {obs}')
    print(f'reward: {reward}')
    print(f'cost: {cost}')
    print(f'terminated: {terminated}')
    print(f'truncated: {truncated}')
    print('*' * 20)
    if terminated or truncated:
        break

--------------------
obs: tensor([-0.1833,  0.3815,  0.4920])
reward: 8444.21875
cost: 7579.5439453125
terminated: False
truncated: False
********************
--------------------
obs: tensor([ 0.7797, -0.6080, -0.4862])
reward: 2589.167236328125
cost: 5112.7470703125
terminated: False
truncated: False
********************
--------------------
obs: tensor([-0.5981, -0.4453, -0.0885])
reward: 7837.98583984375
cost: 3033.127197265625
terminated: False
truncated: False
********************
--------------------
obs: tensor([-0.1391, -0.2324,  0.0532])
reward: 5833.8203125
cost: 9081.12890625
terminated: False
truncated: False
********************
--------------------
obs: tensor([ 0.8725,  0.7580, -0.3909])
reward: 2818.37841796875
cost: 7558.0419921875
terminated: False
truncated: False
********************
--------------------
obs: tensor([-0.5401, -0.5251,  0.2033])
reward: 2505.0634765625
cost: 9097.4619140625
terminated: True
truncated: False
********************


恭喜你！已经成功完成了基础的环境定义，接下来，我们将介绍如何将该环境注册入OmniSafe中，并实现环境参数传递、交互信息记录、算法训练以及结果保存等步骤。

### 定制化环境嵌入

### 快速训练

首先，我们需要将这个环境注册入OmniSafe支持的环境列表中。得益于OmniSafe精心设计的注册机制，我们只需一个装饰器即可实现环境的注册。

In [9]:
@env_register
class ExampleEnv(ExampleEnv):
    pass

如果您后续需要对环境作修改，直接重新注册将会报错。这是因为我们避免产生重名环境冲突。

In [10]:
@env_register
class CustomExampleEnv(ExampleEnv):
    example_configs = 1


env = CustomExampleEnv('Example-v0')
env.example_configs

1

这时，您需要先对环境手动取消注册。

In [11]:
@remove_from_register
class CustomExampleEnv(ExampleEnv):
    pass

之后，您就可以重新注册该环境了。在本教程中，我们会同时嵌套 `env_register` 和 `remove_from_register` 装饰器，这是为了避免环境重复注册造成报错，即确保该环境只被注册一次，以便用户在阅读本教程时多次修改与运行代码。

In [12]:
@env_register
@remove_from_register
class CustomExampleEnv(ExampleEnv):
    example_configs = 2


env = CustomExampleEnv('Example-v0')
env.example_configs

CustomExampleEnv has not been registered yet


2

随后，你可以使用OmniSafe中的算法来训练这个自定义环境。

In [13]:
custom_cfgs = {
    'train_cfgs': {
        'total_steps': 30,
    },
    'algo_cfgs': {
        'steps_per_epoch': 10,
        'update_iters': 1,
    },
}
agent = omnisafe.Agent('PPOLag', 'Example-v0', custom_cfgs=custom_cfgs)
agent.learn()

Loading PPOLag.yaml from /home/jiayi/omnisafe_zjy/omnisafe/utils/../configs/on-policy/PPOLag.yaml


/home/jiayi/miniconda3/envs/omnisafe/lib/python3.8/site-packages/torch/optim/lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


Logging data to ./runs/PPOLag-{Example-v0}/seed-000-2024-03-26-23-04-34/progress.csv

Save with config in config.json

INFO: Start training

...

Warning: trajectory cut off when rollout by epoch at 10.0 steps.

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Metrics                        ┃ Value                  ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ Metrics/EpRet                  │ 28129.7109375          │
│ Metrics/EpCost                 │ 34804.609375           │
│ Metrics/EpLen                  │ 5.0                    │
│ Train/Epoch                    │ 1.0                    │
│ Train/Entropy                  │ 1.4186384677886963     │
│ Train/KL                       │ 0.00010796579590532929 │
│ Train/StopIter                 │ 1.0                    │
│ Train/PolicyRatio/Mean         │ 0.9999998807907104     │
│ Train/PolicyRatio/Min          │ 0.9999998807907104     │
│ Train/PolicyRatio/Max          │ 0.9999998807907104     │
│ Train/PolicyRatio/Std          │ 0.0                    │
│ Train/LR                       │ 9.999999747378752e-05  │
│ Train/PolicyStd                │ 0.9997000694274902     │
│ TotalEnvSteps                  │ 20.0                   │
│ Loss/Loss_pi                   │ 0.0001464843808207661  │
│ Loss/Loss_pi/Delta             │ 0.0001541137753520161  │
│ Value/Adv                      │ -8.940696716308594e-08 │
│ Loss/Loss_reward_critic        │ 947240256.0            │
│ Loss/Loss_reward_critic/Delta  │ 688582112.0            │
│ Value/reward                   │ 0.007712819613516331   │
│ Loss/Loss_cost_critic          │ 615220288.0            │
│ Loss/Loss_cost_critic/Delta    │ 127556928.0            │
│ Value/cost                     │ 0.1568843275308609     │
│ Time/Total                     │ 0.07556390762329102    │
│ Time/Rollout                   │ 0.015428543090820312   │
│ Time/Update                    │ 0.010138273239135742   │
│ Time/Epoch                     │ 0.025595903396606445   │
│ Time/FPS                       │ 390.69842529296875     │
│ Metrics/LagrangeMultiplier/Mea │ 0.07099999487400055    │
│ Metrics/LagrangeMultiplier/Min │ 0.07099999487400055    │
│ Metrics/LagrangeMultiplier/Max │ 0.07099999487400055    │
│ Metrics/LagrangeMultiplier/Std │ 0.0                    │
└────────────────────────────────┴────────────────────────┘

Warning: trajectory cut off when rollout by epoch at 9.0 steps.

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Metrics                        ┃ Value                  ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ Metrics/EpRet                  │ 21468.029296875        │
│ Metrics/EpCost                 │ 25004.017578125        │
│ Metrics/EpLen                  │ 3.6666667461395264     │
│ Train/Epoch                    │ 2.0                    │
│ Train/Entropy                  │ 1.4184980392456055     │
│ Train/KL                       │ 2.7003712602891028e-05 │
│ Train/StopIter                 │ 1.0                    │
│ Train/PolicyRatio/Mean         │ 1.0                    │
│ Train/PolicyRatio/Min          │ 1.0                    │
│ Train/PolicyRatio/Max          │ 1.0                    │
│ Train/PolicyRatio/Std          │ 0.0                    │
│ Train/LR                       │ 0.0                    │
│ Train/PolicyStd                │ 0.9995596408843994     │
│ TotalEnvSteps                  │ 30.0                   │
│ Loss/Loss_pi                   │ 2.441406286379788e-05  │
│ Loss/Loss_pi/Delta             │ -0.0001220703179569682 │
│ Value/Adv                      │ 3.5762788286319847e-08 │
│ Loss/Loss_reward_critic        │ 864482816.0            │
│ Loss/Loss_reward_critic/Delta  │ -82757440.0            │
│ Value/reward                   │ 0.020163696259260178   │
│ Loss/Loss_cost_critic          │ 714492288.0            │
│ Loss/Loss_cost_critic/Delta    │ 99272000.0             │
│ Value/cost                     │ 0.09646058082580566    │
│ Time/Total                     │ 0.11941838264465332    │
│ Time/Rollout                   │ 0.01572561264038086    │
│ Time/Update                    │ 0.010321855545043945   │
│ Time/Epoch                     │ 0.02607893943786621    │
│ Time/FPS                       │ 383.4546813964844      │
│ Metrics/LagrangeMultiplier/Mea │ 0.10524892807006836    │
│ Metrics/LagrangeMultiplier/Min │ 0.10524892807006836    │
│ Metrics/LagrangeMultiplier/Max │ 0.10524892807006836    │
│ Metrics/LagrangeMultiplier/Std │ 0.0                    │
└────────────────────────────────┴────────────────────────┘

(21468.029296875, 25004.017578125, 3.6666667461395264)

干得不错！我们已经完成了这个定制化环境的嵌入和训练。接下来，我们将进一步研究如何为环境指定超参数。

### 参数设定

我们从一个新的示例环境出发，假设这个环境需要传入一个名为 `num_agents` 的参数。我们将展示如何不修改OmniSafe的代码来完成参数设定。

In [14]:
@env_register
@remove_from_register
class NewExampleEnv(ExampleEnv):  # 创造一个新环境
    _support_envs: ClassVar[list[str]] = ['NewExample-v0', 'NewExample-v1']
    num_agents: ClassVar[int] = 1

    def __init__(self, env_id: str, **kwargs) -> None:
        super(NewExampleEnv, self).__init__(env_id, **kwargs)
        self.num_agents = kwargs.get('num_agents', 1)

NewExampleEnv has not been registered yet


此时，`num_agents` 参数为预设值：`1`。

In [15]:
new_env = NewExampleEnv('NewExample-v0')
new_env.num_agents

1

下面我们将展示如何通过 OmniSafe 的接口对该参数进行修改并训练：

In [16]:
custom_cfgs.update({'env_cfgs': {'num_agents': 2}})
agent = omnisafe.Agent('PPOLag', 'NewExample-v0', custom_cfgs=custom_cfgs)
agent.agent._env._env.num_agents

Loading PPOLag.yaml from /home/jiayi/omnisafe_zjy/omnisafe/utils/../configs/on-policy/PPOLag.yaml
{'num_agents': 2}


Logging data to ./runs/PPOLag-{NewExample-v0}/seed-000-2024-03-26-23-04-50/progress.csv

Save with config in config.json

2

非常好！我们将 `num_agents` 设置为了2。这表示我们在未修改代码的情形下成功实现了超参数设定。

### 训练信息记录

在运行训练代码时，你可能已经发现 OmniSafe 通过 `Logger` 记录了训练信息，例如：

```bash
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Metrics                        ┃ Value                   ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ Metrics/EpRet                  │ 2.046875                │
│ Metrics/EpCost                 │ 2.89453125              │
│ Metrics/EpLen                  │ 3.25                    │
│ Train/Epoch                    │ 3.0                     │
...
```
那么我们可否将环境之中的信息输出到日志中呢？答案是肯定的，而且这个过程同样不需要修改OmniSafe的代码。只需要完成两个接口：
1. 在 `__init__` 函数中，将需要输出的信息添加到`self.env_spec_log`中。
2. 实例化 `spec_log` 函数，记录所需的信息。

**请注意：** 目前OmniSafe仅支持在每一个epoch结束时记录这些信息，而不支持在每一个step结束时记录。

In [17]:
@env_register
@remove_from_register
class NewExampleEnv(ExampleEnv):
    _support_envs: ClassVar[list[str]] = ['NewExample-v0', 'NewExample-v1']

    # 定义需要记录的信息
    def __init__(self, env_id: str, **kwargs) -> None:
        super(NewExampleEnv, self).__init__(env_id, **kwargs)
        self.env_spec_log = {'Env/Success_counts': 0}

    # 通过step函数，更新环境信息
    def step(self, action):
        obs, reward, cost, terminated, truncated, info = super().step(action)
        success = int(reward > cost)
        self.env_spec_log['Env/Success_counts'] += success
        return obs, reward, cost, terminated, truncated, info

    # 在logger中记录环境信息
    def spec_log(self, logger) -> dict[str, Any]:
        logger.store({'Env/Success_counts': self.env_spec_log['Env/Success_counts']})
        self.env_spec_log['Env/Success_counts'] = 0

接下来，我们简单训练观察该信息是否被成功记录。

In [18]:
custom_cfgs.update({'train_cfgs': {'total_steps': 10}})
agent = omnisafe.Agent('PPOLag', 'NewExample-v0', custom_cfgs=custom_cfgs)
agent.learn()

Loading PPOLag.yaml from /home/jiayi/omnisafe_zjy/omnisafe/utils/../configs/on-policy/PPOLag.yaml
{'num_agents': 2}


Logging data to ./runs/PPOLag-{NewExample-v0}/seed-000-2024-03-26-23-04-59/progress.csv

Save with config in config.json

INFO: Start training

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Metrics                        ┃ Value                  ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ Metrics/EpRet                  │ 28129.7109375          │
│ Metrics/EpCost                 │ 34804.609375           │
│ Metrics/EpLen                  │ 5.0                    │
│ Train/Epoch                    │ 0.0                    │
│ Train/Entropy                  │ 1.4189385175704956     │
│ Train/KL                       │ 0.00019391297246329486 │
│ Train/StopIter                 │ 1.0                    │
│ Train/PolicyRatio/Mean         │ 1.0                    │
│ Train/PolicyRatio/Min          │ 1.0                    │
│ Train/PolicyRatio/Max          │ 1.0                    │
│ Train/PolicyRatio/Std          │ 0.0                    │
│ Train/LR                       │ 0.0                    │
│ Train/PolicyStd                │ 1.0                    │
│ TotalEnvSteps                  │ 10.0                   │
│ Loss/Loss_pi                   │ 5.035400317865424e-05  │
│ Loss/Loss_pi/Delta             │ 5.035400317865424e-05  │
│ Value/Adv                      │ -6.854534007061375e-08 │
│ Loss/Loss_reward_critic        │ 258657840.0            │
│ Loss/Loss_reward_critic/Delta  │ 258657840.0            │
│ Value/reward                   │ -0.010763613507151604  │
│ Loss/Loss_cost_critic          │ 487662016.0            │
│ Loss/Loss_cost_critic/Delta    │ 487662016.0            │
│ Value/cost                     │ 0.197834774851799      │
│ Time/Total                     │ 0.04369854927062988    │
│ Time/Rollout                   │ 0.0246431827545166     │
│ Time/Update                    │ 0.017728805541992188   │
│ Time/Epoch                     │ 0.04240918159484863    │
│ Time/FPS                       │ 235.80064392089844     │
│ Env/Success_counts             │ 1.5                    │
│ Metrics/LagrangeMultiplier/Mea │ 0.03599999099969864    │
│ Metrics/LagrangeMultiplier/Min │ 0.03599999099969864    │
│ Metrics/LagrangeMultiplier/Max │ 0.03599999099969864    │
│ Metrics/LagrangeMultiplier/Std │ 0.0                    │
└────────────────────────────────┴────────────────────────┘

(28129.7109375, 34804.609375, 5.0)

漂亮！上述代码将在终端输出了环境特化的信息 `Env/Success_counts`。这一过程并不需要对原代码作出改动。

## 总结
安全强化学习是为了AI朝着更鲁棒与安全方向发展的重要研究方向。OmniSafe希望能够成为安全强化学习的有用开源工具包，为AI安全研究提供良好的基础。我们将持续完善OmniSafe的环境定制化接口，使OmniSafe能够更好的适应各种安全强化学习任务。